In [7]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [9]:
# Test dataset is for LINK (1 hr granularity w/ rolling regressions against BTC)

df = pandas.read_csv(f"coinbase-data\data\Link1HrBtcEth.csv", header=None)
ds = df.values
Y = ds[:,0] # predict val of first column (0 or 1)
X = ds[:,1:].astype(float) # using subsequent columns

X

array([[-0.00637546,  0.8971718 , -0.0181168 , ...,  0.06050306,
         0.11272093,  0.12410856],
       [-0.00298787, -0.16882044, -0.0159872 , ...,  0.06890805,
         0.12035003,  0.13051736],
       [-0.00751988,  0.34904343, -0.01529443, ...,  0.07494702,
         0.12545774,  0.13428593],
       ...,
       [-0.06432521, -0.66435556, -0.06017941, ..., -0.10705231,
        -0.09267069, -0.081144  ],
       [ 0.00297325,  0.01748459, -0.01532164, ..., -0.1030366 ,
        -0.08796961, -0.07601299],
       [-0.05012079,  2.4756784 , -0.02570456, ..., -0.10919355,
        -0.09349419, -0.08121545]])

In [10]:
# baseline model (btc and eth)
def create_baseline(xds):
	dim = xds.shape[1]
	# create model
	model = Sequential()
	model.add(Dense(dim, input_dim=dim, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	#model.compile(loss='poisson', optimizer='adam', metrics=['accuracy'])
	return model


In [ ]:
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline(X), epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# Baseline: 91.67% (0.03%)